In [1]:
from diffusers import StableDiffusionXLImg2ImgPipeline
from PIL import Image
from datetime import datetime
import torch
import os
from huggingface_hub import login
from google.colab import userdata  # Access secret tokens safely
from google.colab import drive
drive.mount('/content/drive')

# Fetch secret securely
hf_token = userdata.get('HF_TOKEN')

if hf_token is None:
    raise ValueError("HF_TOKEN secret not found. Please add it via the Colab Secrets tab.")



pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda" if torch.cuda.is_available() else "cpu")


login(token=hf_token)
pipe.enable_attention_slicing()
pipe.enable_model_cpu_offload()
print("✅ SDXL model loaded and ready!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ SDXL model loaded and ready!


In [ ]:

# 📦 Paths
SOURCE_DIR = "/content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples"
DEST_DIR = "/content/drive/MyDrive/Colab_Notebooks/gen_AI_counterfeit"
LOG_FILE = "/content/drive/MyDrive/Colab_Notebooks/corruption_log_img2img.csv"
os.makedirs(DEST_DIR, exist_ok=True)

# 💬 Corruption Prompt List
corruptions = [
    "barcode misaligned",
    "logo slightly faded",
    "batch number mildly obscured",
    "expiry font distorted",
    "dosage line typo",
    "edge blur near label",
    "font mismatch in product name",
    "packaging color altered"
]

def generate_fake(image_path):
    base_image = Image.open(image_path).convert("RGB").resize((512, 512))
    corruption_prompt = f"Fake packaging with {corruptions[torch.randint(0, len(corruptions), (1,)).item()]}"
    print(f"  ↳ Prompt: {corruption_prompt}")

    result = pipe(
        image=base_image,
        prompt=corruption_prompt,
        strength=0.25,
        guidance_scale=7.5,
        num_inference_steps=30
    )

    if result and hasattr(result, "images") and result.images:
        return result.images[0], corruption_prompt
    else:
        raise ValueError("Generation failed — no image returned by SDXL.")

# 🪵 Logging Setup
log_entries = []

# 🚀 Main Loop
print("🎬 Starting img2img counterfeit generation...\n")
for i, filename in enumerate(os.listdir(SOURCE_DIR), 1):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        print(f"[{i}] Processing: {filename}")
        src_path = os.path.join(SOURCE_DIR, filename)

        try:
            src_path = os.path.join(SOURCE_DIR, filename)
            print(f"Attempting to load: {src_path}")
            Image.open(src_path).verify()
            # Step 1: Generate fake based on authentic image
            fake_image, corruption_text = generate_fake(src_path)
            new_filename = f"fake_{filename}"
            fake_image.save(os.path.join(DEST_DIR, new_filename))
            print(f"  ✅ Saved: {new_filename}\n")

            log_entries.append({
                "original": filename,
                "fake": new_filename,
                "corruption": corruption_text,
                "method": "img2img",
                "timestamp": datetime.now().isoformat()
            })

        except Exception as e:
            print(f"  ❌ Error processing {filename}: {e}\n")

# 💾 Save Log File
pd.DataFrame(log_entries).to_csv(LOG_FILE, index=False)
print(f"📁 Log saved to {LOG_FILE} — img2img pipeline complete!\n")

🎬 Starting img2img counterfeit generation...

[1] Processing: Levetiracetam_jpeg_jpg.rf.fc615670270e72e24a18a712658d39a9.jpg
Attempting to load: /content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples/Levetiracetam_jpeg_jpg.rf.fc615670270e72e24a18a712658d39a9.jpg
  ↳ Prompt: Fake packaging with batch number mildly obscured


  0%|          | 0/7 [00:00<?, ?it/s]

  ✅ Saved: fake_Levetiracetam_jpeg_jpg.rf.fc615670270e72e24a18a712658d39a9.jpg

[2] Processing: MTA-36568239_avocel_avocel-5-mg-box-30-tablet_full02_jpg.rf.74c1bb7d4d8db4fa467568ff4f49ee43.jpg
Attempting to load: /content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples/MTA-36568239_avocel_avocel-5-mg-box-30-tablet_full02_jpg.rf.74c1bb7d4d8db4fa467568ff4f49ee43.jpg
  ↳ Prompt: Fake packaging with dosage line typo


  0%|          | 0/7 [00:00<?, ?it/s]

  ✅ Saved: fake_MTA-36568239_avocel_avocel-5-mg-box-30-tablet_full02_jpg.rf.74c1bb7d4d8db4fa467568ff4f49ee43.jpg

[3] Processing: Lorazepam_jpeg_jpg.rf.190f642b4a64181e5e4a9f799e52240f.jpg
Attempting to load: /content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples/Lorazepam_jpeg_jpg.rf.190f642b4a64181e5e4a9f799e52240f.jpg
  ↳ Prompt: Fake packaging with logo slightly faded


...

  0%|          | 0/7 [00:00<?, ?it/s]

  ✅ Saved: fake_MTA-36850255_cefat_cefat_500_mg_tablet_full01_pm8dntb5_jpg.rf.8675dd26a9c0bc7119a9798b3c284594.jpg

[58] Processing: MTA-37305009_oem_placta_75_mg_box_30_tablet_full01_s2dknepu_jpg.rf.e573fa45f4e70d4de3ad131e5ee69403.jpg
Attempting to load: /content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples/MTA-37305009_oem_placta_75_mg_box_30_tablet_full01_s2dknepu_jpg.rf.e573fa45f4e70d4de3ad131e5ee69403.jpg
  ↳ Prompt: Fake packaging with logo slightly faded


  0%|          | 0/7 [00:00<?, ?it/s]

  ✅ Saved: fake_MTA-37305009_oem_placta_75_mg_box_30_tablet_full01_s2dknepu_jpg.rf.e573fa45f4e70d4de3ad131e5ee69403.jpg

[59] Processing: MTA-36602050_aspilets_aspilets_80_mg_box_100_tablet_full01_u8yqy80v_jpg.rf.24c221f77dd64c12449d96bf42cfcf28.jpg
Attempting to load: /content/drive/MyDrive/Colab_Notebooks/new_set_authentic_samples/MTA-36602050_aspilets_aspilets_80_mg_box_100_tablet_full01_u8yqy80v_jpg.rf.24c221f77dd64c12449d96bf42cfcf28.jpg
  ↳ Prompt: Fake packaging with batch number mildly obscured


  0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
print(os.listdir(SOURCE_DIR))

['Levetiracetam_jpeg_jpg.rf.fc615670270e72e24a18a712658d39a9.jpg', 'MTA-36568239_avocel_avocel-5-mg-box-30-tablet_full02_jpg.rf.74c1bb7d4d8db4fa467568ff4f49ee43.jpg', 'Lorazepam_jpeg_jpg.rf.190f642b4a64181e5e4a9f799e52240f.jpg', 'MTA-35779748_no-brand_no-brand_full01_jpg.rf.50818f249894520d10d8720f579b6579.jpg', 'MTA-36596870_dexa-m_dexa_m_0-75_mg_box_500_tablet_full01_dk1ickt4_jpg.rf.d0b0c1dba978782cfdbb944013714406.jpg', 'Lanzofutal_jpg.rf.79a6bb7e2b09f39bc500d936869eb39c.jpg', 'MTA-35779820_no-brand_no-brand_full01_jpg.rf.6ab1f14c699c14ab3edd426b71ef4dff.jpg', 'Levothyroxin_jpeg_jpg.rf.e61fcb0ab37beea7483ad12a933dffbe.jpg', 'Losartan_jpeg_jpg.rf.06f94178ada400f4c48217c46b57dd92.jpg', 'MTA-36557977_alluric_alluric-300-mg-box-50-tablet_full01_jpg.rf.f53b47b5f992d9f2968d77cb3681a5bf.jpg', 'Levodopa_jpeg_jpg.rf.c31bdf8a6ca0eaa9d142006c9c8c9b8d.jpg', 'MTA-36601291_alluric_alluric_300_mg_box_50_tablet_full01_j3sc6qvg_jpg.rf.48915a9ae7ea39041454d869e4f6f268.jpg', 'Lovastatin_jpg.rf.66342a0